# 构建抽取链

在本教程中，我们将使用[工具调用](/docs/concepts/tool_calling)的[聊天模型](/docs/concepts/chat_models)功能，从非结构化文本中抽取结构化信息。我们还将演示如何在此场景下使用[少样本提示](/docs/concepts/few_shot_prompting/)来提升效果。

:::important
本教程需要 `langchain-core>=0.3.20`，且仅适用于支持**工具调用**的模型。
:::

## 环境准备

### Jupyter Notebook

本教程及其他教程推荐在[Jupyter notebooks](https://jupyter.org/)中运行。在交互式环境中学习指南有助于更好地理解内容。安装方法见[这里](https://jupyter.org/install)。

### 安装

安装 LangChain：

import Tabs from '@theme/Tabs';
import TabItem from '@theme/TabItem';
import CodeBlock from "@theme/CodeBlock";

<Tabs>
  <TabItem value="pip" label="Pip" default>
    <CodeBlock language="bash">pip install --upgrade langchain-core</CodeBlock>
  </TabItem>
  <TabItem value="conda" label="Conda">
    <CodeBlock language="bash">conda install langchain-core -c conda-forge</CodeBlock>
  </TabItem>
</Tabs>



更多细节见[安装指南](/docs/how_to/installation)。

### LangSmith

你用 LangChain 构建的许多应用会包含多步、多次 LLM 调用。
随着应用变得越来越复杂，能够检查链或 agent 内部发生了什么变得至关重要。
最好的方式是使用 [LangSmith](https://smith.langchain.com)。

在上面链接注册后，记得设置环境变量以开始记录追踪：

```shell
export LANGSMITH_TRACING="true"
export LANGSMITH_API_KEY="..."
```

或者在 notebook 中，可以这样设置：

```python
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()
```

## 数据结构（Schema）

首先，我们需要描述希望从文本中抽取哪些信息。

我们将使用 Pydantic 定义一个用于抽取个人信息的示例 schema。

In [2]:
from typing import Optional

from pydantic import BaseModel, Field


class Person(BaseModel):
    """Information about a person."""

    # ^ Doc-string for the entity Person.
    # This doc-string is sent to the LLM as the description of the schema Person,
    # and it can help to improve extraction results.

    # Note that:
    # 1. Each field is an `optional` -- this allows the model to decline to extract it!
    # 2. Each field has a `description` -- this description is used by the LLM.
    # Having a good description can help improve extraction results.
    name: Optional[str] = Field(default=None, description="The name of the person")
    hair_color: Optional[str] = Field(
        default=None, description="The color of the person's hair if known"
    )
    height_in_meters: Optional[str] = Field(
        default=None, description="Height measured in meters"
    )

定义 schema 时有两个最佳实践：

1. 注释好**属性**和**schema 本身**：这些信息会发送给 LLM，有助于提升信息抽取的质量。
2. 不要强制 LLM 编造信息！上面我们用 `Optional` 让 LLM 在不知道答案时可以输出 `None`。

:::important
为了获得最佳效果，请详细注释 schema，并确保模型在文本中没有可抽取信息时不被强制返回结果。
:::

## 抽取器（Extractor）

接下来我们用上面定义的 schema 创建一个信息抽取器。

In [3]:
from typing import Optional

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from pydantic import BaseModel, Field

# Define a custom prompt to provide instructions and any additional context.
# 1) You can add examples into the prompt template to improve extraction quality
# 2) Introduce additional parameters to take context into account (e.g., include metadata
#    about the document from which the text was extracted.)
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked to extract, "
            "return null for the attribute's value.",
        ),
        # Please see the how-to about improving performance with
        # reference examples.
        # MessagesPlaceholder('examples'),
        ("human", "{text}"),
    ]
)

我们需要使用支持函数/工具调用的模型。

请查阅[文档](/docs/concepts/tool_calling)，了解所有可用于此 API 的模型。

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />

In [4]:
# | output: false
# | echo: false

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [7]:
structured_llm = llm.with_structured_output(schema=Person)

让我们测试一下：

In [8]:
text = "Alan Smith is 6 feet tall and has blond hair."
prompt = prompt_template.invoke({"text": text})
structured_llm.invoke(prompt)

Person(name='Alan Smith', hair_color='blond', height_in_meters='1.83')

:::important 

抽取是生成式的 🤯

LLM 是生成式模型，所以它们可以完成一些很酷的事情，比如正确地将一个人的身高从英尺转换为米！
:::

[LangSmith 追踪链接](https://smith.langchain.com/public/44b69a63-3b3b-47b8-8a6d-61b46533f015/r)展示了发送给模型的消息序列、调用的工具和其他元数据的完整过程。你可以在[聊天模型部分的追踪](https://smith.langchain.com/public/44b69a63-3b3b-47b8-8a6d-61b46533f015/r/dd1f6305-f1e9-4919-bd8f-339d03a12d01)中查看详细信息。

## 多实体抽取

在**大多数情况下**，你应该抽取的是实体列表而不是单个实体。

这可以通过在 Pydantic 中嵌套模型轻松实现。

In [9]:
from typing import List, Optional

from pydantic import BaseModel, Field


class Person(BaseModel):
    """Information about a person."""

    # ^ Doc-string for the entity Person.
    # This doc-string is sent to the LLM as the description of the schema Person,
    # and it can help to improve extraction results.

    # Note that:
    # 1. Each field is an `optional` -- this allows the model to decline to extract it!
    # 2. Each field has a `description` -- this description is used by the LLM.
    # Having a good description can help improve extraction results.
    name: Optional[str] = Field(default=None, description="The name of the person")
    hair_color: Optional[str] = Field(
        default=None, description="The color of the person's hair if known"
    )
    height_in_meters: Optional[str] = Field(
        default=None, description="Height measured in meters"
    )


class Data(BaseModel):
    """Extracted data about people."""

    # Creates a model so that we can extract multiple entities.
    people: List[Person]

:::important
此处的抽取结果可能不够完美。继续阅读以了解如何使用**参考示例**来提升抽取质量，并查看我们的抽取[操作指南](/docs/how_to/#extraction)获取更多细节。
:::

In [10]:
structured_llm = llm.with_structured_output(schema=Data)
text = "My name is Jeff, my hair is black and i am 6 feet tall. Anna has the same color hair as me."
prompt = prompt_template.invoke({"text": text})
structured_llm.invoke(prompt)

Data(people=[Person(name='Jeff', hair_color='black', height_in_meters='1.83'), Person(name='Anna', hair_color='black', height_in_meters=None)])

:::tip
当 schema 支持提取**多个实体**时，如果文本中没有相关信息，它也允许模型通过提供一个空列表来表示**不提取任何实体**。

这通常是一件**好事**！它允许在实体上指定**必需**属性，而不必强制模型检测这个实体。
:::

你可以在[这里](https://smith.langchain.com/public/7173764d-5e76-45fe-8496-84460bd9cdef/r)查看 LangSmith 追踪记录。

## 参考示例（Reference examples）

LLM 应用的行为可以通过[少样本提示](/docs/concepts/few_shot_prompting/)进行引导。对于[聊天模型](/docs/concepts/chat_models/)，这通常表现为一组输入和响应消息对，演示期望的行为。

例如，我们可以用交替的 `user` 和 `assistant` [消息](/docs/concepts/messages/#role)来传达符号的含义：

In [11]:
messages = [
    {"role": "user", "content": "2 🦜 2"},
    {"role": "assistant", "content": "4"},
    {"role": "user", "content": "2 🦜 3"},
    {"role": "assistant", "content": "5"},
    {"role": "user", "content": "3 🦜 4"},
]

response = llm.invoke(messages)
print(response.content)

7


[结构化输出](/docs/concepts/structured_outputs/)通常在底层使用[工具调用](/docs/concepts/tool_calling/)。这通常涉及生成包含工具调用的 [AI 消息](/docs/concepts/messages/#aimessage)，以及包含工具调用结果的[工具消息](/docs/concepts/messages/#toolmessage)。在这种情况下，消息序列应该是什么样的？

不同的[聊天模型提供商](/docs/integrations/chat/)对有效消息序列有不同的要求。有些会接受这样的（重复）消息序列：

- 用户消息
- 带工具调用的 AI 消息
- 带结果的工具消息

其他提供商则需要一个包含某种响应的最终 AI 消息。

LangChain 包含了一个工具函数 [tool_example_to_messages](https://python.langchain.com/api_reference/core/utils/langchain_core.utils.function_calling.tool_example_to_messages.html)，它可以为大多数模型提供商生成有效的序列。它通过仅需要相应工具调用的 Pydantic 表示来简化结构化少样本示例的生成。

让我们试一下。我们可以将输入字符串和期望的 Pydantic 对象的配对转换为可以提供给聊天模型的消息序列。在底层，LangChain 会将工具调用格式化为每个提供商所需的格式。

注意：此版本的 `tool_example_to_messages` 需要 `langchain-core>=0.3.20`。

In [13]:
from langchain_core.utils.function_calling import tool_example_to_messages

examples = [
    (
        "The ocean is vast and blue. It's more than 20,000 feet deep.",
        Data(people=[]),
    ),
    (
        "Fiona traveled far from France to Spain.",
        Data(people=[Person(name="Fiona", height_in_meters=None, hair_color=None)]),
    ),
]


messages = []

for txt, tool_call in examples:
    if tool_call.people:
        # This final message is optional for some providers
        ai_response = "Detected people."
    else:
        ai_response = "Detected no people."
    messages.extend(tool_example_to_messages(txt, [tool_call], ai_response=ai_response))

检查结果，我们可以看到这两个示例对生成了八条消息：

In [12]:
for message in messages:
    message.pretty_print()

================================ Human Message =================================

The ocean is vast and blue. It's more than 20,000 feet deep.
================================== Ai Message ==================================
Tool Calls:
  Data (d8f2e054-7fb9-417f-b28f-0447a775b2c3)
 Call ID: d8f2e054-7fb9-417f-b28f-0447a775b2c3
  Args:
    people: []
================================= Tool Message =================================

You have correctly called this tool.
================================== Ai Message ==================================

Detected no people.
================================ Human Message =================================

Fiona traveled far from France to Spain.
================================== Ai Message ==================================
Tool Calls:
  Data (0178939e-a4b1-4d2a-a93e-b87f665cdfd6)
 Call ID: 0178939e-a4b1-4d2a-a93e-b87f665cdfd6
  Args:
    people: [{'name': 'Fiona', 'hair_color': None, 'height_in_meters': None}]
================================

让我们对比下有无这些示例消息时的表现。例如，传入一条我们期望不抽取任何人的消息：

In [15]:
message_no_extraction = {
    "role": "user",
    "content": "The solar system is large, but earth has only 1 moon.",
}

structured_llm = llm.with_structured_output(schema=Data)
structured_llm.invoke([message_no_extraction])

Data(people=[Person(name='Earth', hair_color='None', height_in_meters='0.00')])

在这个例子中，模型可能会错误地生成人物记录。

由于我们的少样本示例包含了“负例”，因此可以鼓励模型在这种情况下表现正确：

In [16]:
structured_llm.invoke(messages + [message_no_extraction])

Data(people=[])

:::tip

[LangSmith](https://smith.langchain.com/public/b3433f57-7905-4430-923c-fed214525bf1/r) 的追踪可以展示本次运行发送给聊天模型的消息序列、生成的工具调用、延迟、token 数等详细元数据。

:::

更多关于带参考示例的抽取工作流细节，包括如何结合提示模板和自定义示例消息生成，见[本指南](/docs/how_to/extraction_examples/)。

## 后续步骤

现在你已经了解了 LangChain 抽取的基础知识，可以继续阅读以下操作指南：

- [添加示例](/docs/how_to/extraction_examples)：详细介绍如何用**参考示例**提升效果。
- [处理长文本](/docs/how_to/extraction_long_text)：如果文本超出 LLM 上下文窗口该怎么办？
- [使用解析方法](/docs/how_to/extraction_parse)：对于不支持**工具/函数调用**的模型，如何用提示词方式抽取。